In [1]:
import pandas as pd
from datasets import Dataset
from transformers import RobertaTokenizer
from sklearn.model_selection import train_test_split

In [2]:
# Cargar el dataset etiquetado en un DataFrame
df = pd.read_csv(r"C:\Users\samue\OneDrive\Escritorio\df_bipolar_full_sept7_ok.csv")

In [3]:
df.head()

,text,submission_type,subreddit,labels
0,Media: Nobody knows what kamala is about\n\nMe...,comment,politics,Democrat
1,NYT breaking news that Netanyahu has agreed to...,comment,politics,Democrat
2,I was thinking this morning about how freaking...,comment,politics,Democrat
3,I was on a break talking to a 25-year old cowo...,comment,politics,Republican
4,Went to the Kamala rally today in WI and it wa...,comment,politics,Democrat


In [4]:
df.shape

(84045, 4)

In [5]:
# Mapeamos las etiquetas de texto a valores numéricos
label_mapping = {"Republican": 0, "Democrat": 1}

df['labels'] = df['labels'].map(label_mapping)

In [6]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['labels'], random_state=42)

In [7]:
# Convertir a Dataset compatible con Hugging Face
from datasets import Dataset

# Guardar las etiquetas reales antes de hacer el drop
true_labels = test_df['labels'].values

# Eliminar la columna 'labels' para preparar el test sin las etiquetas
test_features = test_df.drop(columns=['labels'])


train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_features)

In [8]:
# Cargar el tokenizador de RoBERTa
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenizar los datos
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=512)

# Tokenizar los conjuntos de entrenamiento y prueba
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

c:\Users\samue\anaconda3\envs\project_g\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/67236 [00:00<?, ? examples/s]

Map:   0%|          | 0/16809 [00:00<?, ? examples/s]

In [9]:
# Guardar el tokenizador utilizado
tokenizer.save_pretrained(r"C:\Users\samue\OneDrive\Escritorio\Docs\4GeeksAcademy\FINAL_PROJECT\4geeks_finalproject\src/tokenizer")

('C:\\Users\\samue\\OneDrive\\Escritorio\\Docs\\4GeeksAcademy\\FINAL_PROJECT\\4geeks_finalproject\\src/tokenizer\\tokenizer_config.json',
 'C:\\Users\\samue\\OneDrive\\Escritorio\\Docs\\4GeeksAcademy\\FINAL_PROJECT\\4geeks_finalproject\\src/tokenizer\\special_tokens_map.json',
 'C:\\Users\\samue\\OneDrive\\Escritorio\\Docs\\4GeeksAcademy\\FINAL_PROJECT\\4geeks_finalproject\\src/tokenizer\\vocab.json',
 'C:\\Users\\samue\\OneDrive\\Escritorio\\Docs\\4GeeksAcademy\\FINAL_PROJECT\\4geeks_finalproject\\src/tokenizer\\merges.txt',
 'C:\\Users\\samue\\OneDrive\\Escritorio\\Docs\\4GeeksAcademy\\FINAL_PROJECT\\4geeks_finalproject\\src/tokenizer\\added_tokens.json')

In [10]:
# Si también guardaste el tokenizador
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained(r"C:\Users\samue\OneDrive\Escritorio\Docs\4GeeksAcademy\FINAL_PROJECT\4geeks_finalproject\src\tokenizer")


In [11]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

# Cargar el modelo para clasificación de secuencias con 2 etiquetas (Demócrata/Republicano)
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)

# Definir el Trainer con los datos de entrenamiento, que ya incluyen las etiquetas
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

# Entrenar el modelo
trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\samue\anaconda3\envs\project_g\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/25215 [00:00<?, ?it/s]

{'loss': 0.1532, 'grad_norm': 0.06808625906705856, 'learning_rate': 1.9603410668253024e-05, 'epoch': 0.06}
{'loss': 0.0693, 'grad_norm': 0.004451404791325331, 'learning_rate': 1.920682133650605e-05, 'epoch': 0.12}
{'loss': 0.0481, 'grad_norm': 0.03014403209090233, 'learning_rate': 1.8810232004759074e-05, 'epoch': 0.18}
{'loss': 0.038, 'grad_norm': 0.00703075435012579, 'learning_rate': 1.84136426730121e-05, 'epoch': 0.24}
{'loss': 0.0321, 'grad_norm': 0.41191479563713074, 'learning_rate': 1.801705334126512e-05, 'epoch': 0.3}
{'loss': 0.0308, 'grad_norm': 0.0018506153719499707, 'learning_rate': 1.7620464009518146e-05, 'epoch': 0.36}
{'loss': 0.0318, 'grad_norm': 0.3027184009552002, 'learning_rate': 1.722387467777117e-05, 'epoch': 0.42}
{'loss': 0.0239, 'grad_norm': 0.0017950630281120539, 'learning_rate': 1.6827285346024194e-05, 'epoch': 0.48}
{'loss': 0.0179, 'grad_norm': 0.0008981557912193239, 'learning_rate': 1.6430696014277215e-05, 'epoch': 0.54}
{'loss': 0.021, 'grad_norm': 0.0005893

  0%|          | 0/2102 [00:00<?, ?it/s]

{'eval_runtime': 331.1612, 'eval_samples_per_second': 50.758, 'eval_steps_per_second': 6.347, 'epoch': 1.0}
{'loss': 0.0173, 'grad_norm': 0.0010047669056802988, 'learning_rate': 1.3257981360301409e-05, 'epoch': 1.01}
{'loss': 0.0059, 'grad_norm': 0.0012426926987245679, 'learning_rate': 1.2861392028554434e-05, 'epoch': 1.07}
{'loss': 0.0029, 'grad_norm': 0.0002543394803069532, 'learning_rate': 1.2464802696807456e-05, 'epoch': 1.13}
{'loss': 0.0063, 'grad_norm': 0.0001983623660635203, 'learning_rate': 1.2068213365060482e-05, 'epoch': 1.19}
{'loss': 0.0064, 'grad_norm': 0.0004751285305246711, 'learning_rate': 1.1671624033313505e-05, 'epoch': 1.25}
{'loss': 0.0039, 'grad_norm': 0.00017160511924885213, 'learning_rate': 1.127503470156653e-05, 'epoch': 1.31}
{'loss': 0.0142, 'grad_norm': 0.000745914876461029, 'learning_rate': 1.0878445369819552e-05, 'epoch': 1.37}
{'loss': 0.0088, 'grad_norm': 0.00021610646217595786, 'learning_rate': 1.0481856038072578e-05, 'epoch': 1.43}
{'loss': 0.0068, 'gr

KeyboardInterrupt: 

In [41]:
# Evaluar el modelo
results = trainer.evaluate()

print(results)

  0%|          | 0/2102 [00:00<?, ?it/s]

{'eval_runtime': 300.9307, 'eval_samples_per_second': 55.857, 'eval_steps_per_second': 6.985, 'epoch': 3.0}


In [55]:
# Guardar el modelo entrenado
trainer.save_model(r"C:\Users\samue\OneDrive\Escritorio\Docs\4GeeksAcademy\FINAL_PROJECT\4geeks_finalproject\src/modelo_entrenado")

In [61]:
from transformers import RobertaForSequenceClassification

# Cargar el modelo entrenado
trainer = RobertaForSequenceClassification.from_pretrained(r"C:\Users\samue\OneDrive\Escritorio\Docs\4GeeksAcademy\FINAL_PROJECT\4geeks_finalproject\src\modelo_entrenado")

In [17]:
# Realizar inferencia sobre el conjunto de test
predictions = trainer.predict(tokenized_test)


In [21]:
# 'predictions' devuelve un objeto con logits (predicciones sin procesar)
logits = predictions.predictions

In [27]:
from scipy.special import softmax

# Convertir los logits en probabilidades
import numpy as np

# Aplicar softmax para convertir los logits en probabilidades
probabilities = softmax(logits, axis=1)

# Obtener las predicciones finales (clase con mayor probabilidad)
predicted_labels = np.argmax(probabilities, axis=1)

# Imprimir los resultados
print(predicted_labels)

[0 0 0 ... 1 0 0]


In [36]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Convertir el conjunto tokenizado a DataFrame para obtener las etiquetas reales
test_df = tokenized_test.to_pandas()

# Calcular la precisión (accuracy)
accuracy = accuracy_score(true_labels, predicted_labels)

# Calcular precision, recall y f1-score
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='binary')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")


Accuracy: 0.9988696531619965
Precision: 0.9982587497823437
Recall: 0.9984326018808778
F1-score: 0.9983456682629517


In [1]:
import torch
from scipy.special import softmax
import numpy as np

# Mover el modelo a la GPU si está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Texto nuevo para probar
new_text = ['what a dangerous with all this guns', "Red is the best"]

# Tokenizar el texto
inputs = tokenizer(new_text, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Mover los tensores a la GPU
inputs = {key: value.to(device) for key, value in inputs.items()}

# Realizar inferencia en el nuevo texto
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Mover logits a la CPU para aplicar softmax
probabilities = softmax(logits.cpu().numpy(), axis=1)

# Obtener las predicciones finales (clase con mayor probabilidad)
predicted_labels = np.argmax(probabilities, axis=1)

# Mapear las etiquetas predichas a sus nombres (0: Republicano, 1: Demócrata)
label_map = {0: "Republicano", 1: "Demócrata"}
predicted_names = [label_map[label] for label in predicted_labels]

# Imprimir las predicciones
print(predicted_names)



NameError: name 'model' is not defined